In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import sys
sys.path.append('/home/chenani/ownCloud/Workspaces/Eclipse/dataAnalysis/Sleep-current/src/')

In [2]:
%matplotlib
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as pl
import pickle as pkl
import fnmatch,os
import itertools
import statsmodels.formula.api as sm
from scipy.stats import pearsonr,ttest_ind,ranksums,linregress
import pandas as pd
import seaborn as sns
from signale.tools import locate
import statsmodels.api as sm
from matplotlib.gridspec import GridSpec
from NeuroTools import analysis
from scipy import stats

Using matplotlib backend: Qt4Agg


/home/chenani/anaconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
/home/chenani/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [3]:
#spk = pkl.load(open('/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3661/LinearTrack_03-04-15Day2/parrentData/FiringUnits-N.spk','rb'))
#R0s = pkl.load(open('/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/Rat3661-03-04-15Day2-theta.R0'))
#Rls = pkl.load(open('/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/Rat3661-03-04-15Day2-theta.Rl'))
#tBins = pkl.load(open('/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3661/LinearTrack_03-04-15Day2/parrentData/thBins.npArr'))
#rippDB = pd.read_pickle('/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/control/Rat3661Control/dataFiles/sleepDB/Rat3661-RipplesTimeDB-SKLRN.pd')
#S1,LT,S2  = [spk.time_slice(item[0],item[-1]) for item in tBins]
clr4 = ["#cb535a","#5ba952","#c97d3d","#9a993e"]
sns.set_style('ticks')

In [5]:
r0List = sorted([os.path.join(pth[0],pth[1]) for pth in locate('*-theta.R0','/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/DATA/')])
rlList = sorted([os.path.join(pth[0],pth[1]) for pth in locate('*-theta.Rl','/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/DATA/')])
rippList = sorted([os.path.join(pth[0],pth[1]) for pth in locate('*RipplesTimeDB-SKLRN.pd','/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/')])
binList = sorted([os.path.join(pth[0],pth[1]) for pth in locate('thBins.npArr','/home/chenani/dataWork/ali/MECLesion_SleepData/')])
lesList = [3656,3754,3837,3838,3903,434,405]
binList

['/home/chenani/dataWork/ali/MECLesion_SleepData/Magda/Rat405Lesion/LinearTrack_0-0-0_Day1/parrentData/thBins.npArr',
 '/home/chenani/dataWork/ali/MECLesion_SleepData/Magda/Rat434Lesion/LinearTrack_0-0-0_Day1/parrentData/thBins.npArr',
 '/home/chenani/dataWork/ali/MECLesion_SleepData/Magda/Rat614Control/LinearTrack_05-31-13_Day1/parrentData/thBins.npArr',
 '/home/chenani/dataWork/ali/MECLesion_SleepData/Magda/Rat614Control/LinearTrack_06-01-13_Day2/parrentData/thBins.npArr',
 '/home/chenani/dataWork/ali/MECLesion_SleepData/Magda/Rat614Control/LinearTrack_06-11-13_Day3/parrentData/thBins.npArr',
 '/home/chenani/dataWork/ali/MECLesion_SleepData/Magda/Rat616Control/LinearTrack_2013-06-02_Day1/parrentData/thBins.npArr',
 '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3656/LinearTrack_03-03-2015_Day1/parrentData/thBins.npArr',
 '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3656/LinearTrack_03-04-2015_Day2/parrentData/thBins.npArr',
 '/home/chenani/dataWork/ali/MECLesion

In [49]:
patCCDF = pd.DataFrame([],columns=['ccdf','anDate','type','group','session','ptIdx'])
sessList = ['PRE','POST']
kk = 0
for kk,rlFile in enumerate(rlList):
    print kk
    Rls = pkl.load(open(rlFile,'rb'))
    fNmae = rlFile.split('/')[-1]
    anName = fNmae.split('-')[0]
    anID = anName.split('t')[-1]
    dday = fNmae.split('-')[-2]
    try:
        anID = int(anID)
        print anID,dday
    except:
        print 'No good file!!!'
        anID = 0
    if anID in lesList:
        group = 'lesion'
    else:
        group = 'control'
    bF = [binfile for binfile in binList if (binfile.find(anName)>0)and(binfile.find(dday)>0)]
    if len(bF):
        print bF[0]
        tBins = pkl.load(open(bF[0],'rb'))
    rippF = [rippfile for rippfile in rippList if (rippfile.find(anName)>0)]
    if len(rippF):
        rippDB = pd.read_pickle(rippF[-1])
        rippDB = rippDB.replace('01-sleep01','PRE')
        rippDB = rippDB.replace('05-sleep05','PRE')
        rippDB = rippDB.replace('03-sleep02','POST')
        rippDB = rippDB.replace('07-sleep06','POST')
        rippDB = rippDB[rippDB.date==dday.lower()]
    if anID:
        tPre  = np.array([(t+tBins[0][ii+1])/2 for ii,t in enumerate(tBins[0][:-1])])
        tPost = np.array([(t+tBins[2][ii+1])/2 for ii,t in enumerate(tBins[2][:-1])])
        swrTstmpPRE = rippDB[(rippDB.session=='PRE')&(rippDB.wc==300)].t_peak.as_matrix()
        swrTstmpPOS = rippDB[(rippDB.session=='POST')&(rippDB.wc==300)].t_peak.as_matrix()
        fgbTstmpPRE = rippDB[(rippDB.session=='PRE')&(rippDB.wc==100)].t_peak.as_matrix()
        fgbTstmpPOS = rippDB[(rippDB.session=='POST')&(rippDB.wc==100)].t_peak.as_matrix()
        tBinCenters = [tPre,tPost]
        swrStamps = [swrTstmpPRE,swrTstmpPOS]
        fgbStamps = [fgbTstmpPRE,fgbTstmpPOS]
        print swrTstmpPRE.size,swrTstmpPOS.size
        for ii in range(1,10):
            for jj in [0,1]:
                pt = Rls[1][jj][10-ii]
                ptPeaks = np.where(pt>(pt.mean()+3*pt.std()))[0]
                ptTimestamps = tBinCenters[jj][ptPeaks][:-1]
                for ptP in ptTimestamps:
                    if swrStamps[jj].size and fgbStamps[jj].size:
                        if (np.abs(swrStamps[jj]-ptP).min()<np.abs(fgbStamps[jj]-ptP).min()):
                            patCCDF = patCCDF.append(pd.DataFrame([[np.abs(swrStamps[jj]-ptP).min(),kk,'swr',group,sessList[jj],ii]],
                                                              columns=['ccdf','anDate','type','group','session','ptIdx']),ignore_index=True)
                        else:
                            patCCDF = patCCDF.append(pd.DataFrame([[np.abs(fgbStamps[jj]-ptP).min(),kk,'fgb',group,sessList[jj],ii]],
                                                              columns=['ccdf','anDate','type','group','session','ptIdx']),ignore_index=True)
                #diff,pred,N = analysis.crosscorrelate(ptTimestamps,swrStamps[jj],lag=500)
                #diffg,pred,N = analysis.crosscorrelate(ptTimestamps,fgbStamps[jj],lag=500)
                #hstSWR,bins= np.histogram(diff,bins=np.arange(-450,501,100),normed=True)
                #hstFGB,bins= np.histogram(diffg,bins=np.arange(-450,501,100),normed=True)
                #if diff.size:
                #    swrSyncPrcnt = 1e2*np.sum(np.abs(diff)<50)/diff.size
                #else:
                #    swrSyncPrcnt = np.nan
                #if diffg.size:
                #    fgbSyncPrcnt = 1e2*np.sum(np.abs(diffg)<50)/diffg.size
                #else:
                #    fgbSyncPrcnt = np.nan
                #if group=='lesion':
                #    pl.scatter(ii+jj/2.0,hstSWR[4]-hstFGB[4],c=clr4[-1*jj])
                    #patCCDF = patCCDF.append(pd.DataFrame([[hstSWR[4]-hstFGB[4],kk,group,sessList[jj],ii]],columns=['ccdf','anDate','group','session','ptIdx']),ignore_index=True)
                #    patCCDF = patCCDF.append(pd.DataFrame([[swrSyncPrcnt,kk,group,sessList[jj],ii]],
                #                                          columns=['ccdf','anDate','group','session','ptIdx']),ignore_index=True)
                #else:
                    #pl.scatter(ii+jj/2.0,hstSWR[4]-hstFGB[4],c=clr4[jj],s=1)
                    #patCCDF = patCCDF.append(pd.DataFrame([[hstSWR[4]-hstFGB[4],kk,group,sessList[jj],ii]],
                    #                                      columns=['ccdf','anDate','group','session','ptIdx']),ignore_index=True)
                #    patCCDF = patCCDF.append(pd.DataFrame([[swrSyncPrcnt,kk,group,sessList[jj],ii]],
                #                                          columns=['ccdf','anDate','group','session','ptIdx']),ignore_index=True)
                                             
                    #print hstSWR
        #pl.plot(hstSWR)

0
3656 Day1
/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3656/LinearTrack_03-03-2015_Day1/parrentData/thBins.npArr
1102 839
1
3656 Day2
/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3656/LinearTrack_03-04-2015_Day2/parrentData/thBins.npArr
814 874
2
3656 Day3
/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3656/LinearTrack_03-05-2015_Day3/parrentData/thBins.npArr
484 403
3
3661 Day1
/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3661/LinearTrack_03-03-15Day1/parrentData/thBins.npArr
945 1065
4
3661 Day2
/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3661/LinearTrack_03-04-15Day2/parrentData/thBins.npArr
1493 1208
5
3661 Day3
/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3661/LinearTrack_03-05-15Day3/parrentData/thBins.npArr
953 1018
6
3754 Day1
/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3754/LinearTrack_07-06-15_Day1/parrentData/thBins.npArr
149 273
7
3754 Day2
/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat

In [111]:
patCCDF.to_pickle('/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/DATA/patCCDF.db')

In [63]:
sns.violinplot(x='session',y='ccdf',data=patCCDF[(patCCDF.ptIdx==1)&(patCCDF.group=='control')])

In [93]:
syncPCdb = pd.DataFrame([],columns=['anID','group','patID','session','hfe_type','prcnt'])
for anID in np.unique(patCCDF.anDate):
    df = patCCDF[patCCDF.anDate==anID]
    for ptID in range(1,10):
        for sess in ['PRE','POST']:
            dfSessID = df[(df.session==sess)&(df.ptIdx==ptID)]
            if dfSessID.shape[0]:
                for typ in ['swr','fgb']:
                    dfTyp = dfSessID[dfSessID['type']==typ]
                    if dfTyp.shape[0]:
                        syncPCdb = syncPCdb.append(pd.DataFrame([[anID,np.unique(dfTyp.group)[0],ptID,sess,typ,
                                                                  1e2*dfTyp[dfTyp.ccdf<50].shape[0]/dfSessID.shape[0]]]
                                                  ,columns=['anID','group','patID','session','hfe_type','prcnt']),ignore_index=True)

In [110]:
f,(ax,bx) = pl.subplots(1,2,sharey=True)
sns.tsplot(time="patID", value="prcnt",unit="anID", condition="group",data=syncPCdb[(syncPCdb.session=='PRE')&(syncPCdb.hfe_type=='swr')],
           interpolate=True,err_style="boot_traces",n_boot=50,estimator=np.nanmean,color=['r','k'],ax=ax)
sns.tsplot(time="patID", value="prcnt",unit="anID", condition="group",data=syncPCdb[(syncPCdb.session=='PRE')&(syncPCdb.hfe_type=='swr')],
           interpolate=False,err_style="ci_bars",estimator=np.nanmean,color=['r','k'],ax=ax)
sns.tsplot(time="patID", value="prcnt",unit="anID", condition="group",data=syncPCdb[(syncPCdb.session=='POST')&(syncPCdb.hfe_type=='swr')],
           interpolate=True,err_style="boot_traces",n_boot=50,estimator=np.nanmean,color=['r','k'],ax=bx)
sns.tsplot(time="patID", value="prcnt",unit="anID", condition="group",data=syncPCdb[(syncPCdb.session=='POST')&(syncPCdb.hfe_type=='swr')],
           interpolate=False,err_style="ci_bars",estimator=np.nanmean,color=['r','k'],ax=bx)

In [50]:
f,(ax,bx) = pl.subplots(1,2,sharey=True)
sns.tsplot(time="ptIdx", value="ccdf",unit="anDate", condition="group",data=patCCDF[patCCDF.session=='PRE'],
           interpolate=True,err_style="boot_traces",n_boot=50,estimator=np.nanmean,ax=ax)
sns.tsplot(time="ptIdx", value="ccdf",unit="anDate", condition="group",data=patCCDF[patCCDF.session=='PRE'],
           interpolate=False,err_style="ci_bars",estimator=np.nanmean,ax=ax)
sns.tsplot(time="ptIdx", value="ccdf",unit="anDate", condition="group",data=patCCDF[patCCDF.session=='POST'],
           interpolate=True,err_style="boot_traces",n_boot=50,estimator=np.nanmean,ax=bx)
sns.tsplot(time="ptIdx", value="ccdf",unit="anDate", condition="group",data=patCCDF[patCCDF.session=='POST'],
           interpolate=False,err_style="ci_bars",estimator=np.nanmean,ax=bx)
#pl.xscale('log')
#pl.xlim(0.5,4.5)

ValueError: Index contains duplicate entries, cannot reshape

In [57]:
np.diff(patCCDF.index).min()


1